In [52]:
import torch
from torch.utils.data import Dataset, DataLoader
import random

In [15]:
# retrieve the embeddings of the sentences
import csv
with open('../result/embedding.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    embedding_dict = [row for row in csv_reader]
    
# how to index the data: emdedding_dict[sentence_id][language_pair]

In [159]:
def get_embedding(embedding_dict, sent_id, prompt_lang, sent_lang):
    """helper method to make indexing the data easier"""
    return embedding_dict[sent_id][f'{prompt_lang}-{sent_lang}']

emb_dim=124 #CHANGE THIS!!! 
# TEMPORARY!!!!!!!!!!!!!!!! (because the embedding dict doesn't work yet)
def get_embedding(embedding_dict, sent_id, prompt_lang, sent_lang):
    return (torch.zeros((emb_dim)))

get_embedding(None, 1,"English", "Russian").shape

torch.Size([124])

In [158]:
get_embedding(embedding_dict, 0, "English", "Russian")
# with real data, this should not be all zeros

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [160]:
languages = ['English', 'Chinese_Simplified', 'Russian', 'Dutch', 'German']
class EmbeddingsDataset(Dataset):
    def __init__(self, embedding_dict, eng_prompt=True):
        """
        embedding_dict: the dict with all the embeddings
        eng_prompt: if true, uses English prompting, otherwise self-prompting
        """
        self.embedding_dict = embedding_dict
        self.eng_prompt=eng_prompt
        
    def __len__(self):
        return len(self.embedding_dict)
    
    def __getitem__(self, idx):
        """
        returns a dataset example of size (2, emb_dim) which represents embedding of 
        the same sentence in two (randomly chosen) different languages
        """
        
        # choose two different languages (this samples without replacement)
        lang_1, lang_2 = random.sample(languages, 2)
        
        # english prompting
        if self.eng_prompt: 
            return torch.stack([
                get_embedding(self.embedding_dict, idx, "English", lang_1),
                get_embedding(self.embedding_dict, idx, "English", lang_2),
            ])
        
        # self-prompting
        else: 
            return torch.stack([
                get_embedding(self.embedding_dict, idx, lang_1, lang_1),
                get_embedding(self.embedding_dict, idx, lang_2, lang_2),
            ])
        
        

In [134]:
# define train,val,test sizes (60%, 20%, 20%)
train_size, val_size, test_size = int(len(embedding_dict)*0.6), int(len(embedding_dict)*0.2), int(len(embedding_dict)*0.2)
eng_prompt = True

# make the datasets
train_set = EmbeddingsDataset(embedding_dict[:train_size], eng_prompt=eng_prompt)
val_set   = EmbeddingsDataset(embedding_dict[train_size:train_size+val_size], eng_prompt=eng_prompt)
test_set  = EmbeddingsDataset(embedding_dict[train_size+val_size:], eng_prompt=eng_prompt)

In [161]:
# make the train, val, and test loader
batch_size = 8

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_set,   batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=True)

In [156]:
# make the MLP and optimizer
mlp = torch.nn.Linear(emb_dim, emb_dim).to("cpu")
optimizer = ...

In [162]:
# training loop: 
device = "cpu"

# how long do we train? Do we look at the val loss to decide? 
# TODO: should add a second loop here

for batch in train_loader:
    # batch.shape: (bs, emb_dim)
    print(batch.shape)
    batch = batch.to(device)
    
    # flatten batch to (2*bs, emb_dim)
    # flattens to (s1a, s1b, s2c, s2d, ...) where s1 is sentence 1, and a,b,... are languages
    batch = batch.flatten(0,1)    
    print(batch.shape) 
    
    
    optimizer.zero_grad()

    
    # pass it through the mlp to get the new embeddings
    outputs = mlp(batch)
    print(outputs.shape)
    
    # apply the contrastive loss to the outputs
    # positive examples same sentence, different languages
    # negative examples: different sentences
    
    # TODO: 
    # loss = .....
    
    # take a step
    loss.backward()
    optimizer.step()
    
    
    with torch.zero_grad():
        # validation 
        # metric = ...
        pass
    
    break # remove this

torch.Size([8, 2, 124])
torch.Size([16, 124])


NameError: name 'optimizer' is not defined